In [91]:
from pypdf import PdfReader
import pandas as pd
import os, re, math
import numpy as np
from scipy.sparse import dok_matrix
from sklearn.preprocessing import normalize
import matplotlib.pyplot as plt

In [95]:
meta_data = pd.read_csv("article-metadata.csv")
meta_data

title  year  journal  \
0   Neural networks and physical systems with emer...  1982     PNAS   
1   Learning representations by back-propagating e...  1986   Nature   
2   ImageNet Classification with Deep Convolutiona...  2012  NeurIPS   
3                                       Deep learning  2015   Nature   
4        Deep Residual Learning for Image Recognition  2016     CVPR   
5                           Attention Is All You Need  2017  NeurIPS   
6   CheXNet: Radiologist-Level Pneumonia Detection...  2017    arXiv   
7            Densely Connected Convolutional Networks  2017     CVPR   
8   Cardiologist-level arrhythmia detection and cl...  2019   Nature   
9   Language Models are Unsupervised Multitask Lea...  2019   OpenAI   
10  CheXpert: A Large Chest Radiograph Dataset wit...  2019      ACM   
11  EfficientNet: Rethinking Model Scaling for Con...  2019     ICML   
12  Domain-Specific Language Model Pretraining for...  2021      ACM   
13  Learning Transferable Visual Models From Natur...  2021     ICML   
14  Data-efficient and weakly supervised computati...  2021   Nature   
15  Accurate prediction of protein structures and ...  2021  Science   
16  Highly accurate protein structure prediction w...  2021   Nature   
17  Evolutionary-scale prediction of atomic-level ...  2023  Science   
18  A visual-language foundation model for computa...  2024   Nature   
19  Towards a general-purpose foundation model for...  2024   Nature   

         topic     short_name  main_pages  
0        model    neural-nets           5  
1        model       backprop           4  
2        model            cnn           8  
3        model  deep-learning           7  
4          vis         resnet           8  
5        model      attention           9  
6    radiology        chexnet           5  
7          vis       densenet           8  
8   cardiology            ecg           6  
9          LLM          gpt-2          10  
10   radiology       chexpert           7  
11         vis   efficientnet           8  
12         LLM     pubmedbert          21  
13         LLM           clip           9  
14   pathology           clam          15  
15     protein    rosettafold           5  
16     protein      alphafold           7  
17     protein          esm-2           7  
18   pathology          conch          10  
19   pathology            uni          10

In [99]:
article_to_main_pages = dict(zip(meta_data["short_name"], meta_data["main_pages"]))

In [100]:
# write a helper function to extract all the text from the pdf
def extract_full_text_from_pdf(pdf_name):
    pdf_path = "articles/" + pdf_name + ".pdf"
    main_pages = article_to_main_pages[pdf_name]
    reader = PdfReader(pdf_path)
    text = ""   
    for page in reader.pages[0: main_pages]:
        raw_text = page.extract_text()
        text += raw_text
    return text

# extract all the text from the pdfs stored at `articles/` and organize them into a dict
docname_to_ls_of_strs = {}
for pdf_name in article_to_main_pages.keys():
    raw_text = extract_full_text_from_pdf(pdf_name)
    # remove all whitespace, numbers, and special characters, and convert to lowercase
    # TODO: clean this up later on
    cleaned_ls_of_strs = [re.sub(r'(^-+|-+$)', '', re.sub(r'[^\w\s-]|[\d]', '', term)).lower() for term in raw_text.split()]
    cleaned_ls_of_strs = [term for term in cleaned_ls_of_strs if term != ""]
    docname_to_ls_of_strs[pdf_name] = cleaned_ls_of_strs

docname_to_ls_of_strs

{'neural-nets': ['proc',
  'natl',
  'acad',
  'sci',
  'usavol',
  'pp',
  'april',
  'biophysics',
  'neural',
  'networks',
  'and',
  'physical',
  'systems',
  'with',
  'emergent',
  'collective',
  'computational',
  'abilitiesassociative',
  'memoryparallel',
  'processingcategorizationcontent-addressable',
  'memoryfail-soft',
  'devices',
  'j',
  'j',
  'hopfield',
  'division',
  'of',
  'chemistry',
  'and',
  'biology',
  'california',
  'institute',
  'of',
  'technology',
  'pasadena',
  'california',
  'and',
  'bell',
  'laboratories',
  'murray',
  'hill',
  'new',
  'jersey',
  'contributed',
  'by',
  'john',
  'j',
  'hopfweld',
  'january',
  'abstract',
  'computational',
  'properties',
  'of',
  'use',
  'to',
  'biological',
  'or',
  'ganisms',
  'or',
  'to',
  'the',
  'construction',
  'of',
  'computers',
  'can',
  'emerge',
  'as',
  'col',
  'lective',
  'properties',
  'of',
  'systems',
  'having',
  'a',
  'large',
  'number',
  'of',
  'simple',
 

In [101]:
def compute_tf_for_a_doc(doc_ls_of_strs: list[str]) -> dict:
    '''compute the term frequency of each term in the document and return a dict of the counts'''
    total_terms = len(doc_ls_of_strs)
    term_to_tf = {}
    for term in doc_ls_of_strs:
        term_to_tf[term] = term_to_tf.get(term, 0) + 1
    
    for term in term_to_tf:
        term_to_tf[term] /= total_terms
    return term_to_tf

In [102]:
def compute_tf_idf_for_a_doc(current_doc_tf_dict: dict, other_docs_ls_of_dict: list[dict]) -> dict:
    '''
    compute the tf-idf of each term in the document and return a dict 
    idf = log(number of documents / number of documents that contain the word)
    '''
    n_total_docs = len(other_docs_ls_of_dict) + 1
    current_doc_tf_idf_dict = {}
    for term in current_doc_tf_dict:
        n_docs_with_term = 1 + sum(1 for other_doc in other_docs_ls_of_dict if term in other_doc)
        current_doc_tf_idf_dict[term] = current_doc_tf_dict[term] * math.log(n_total_docs / n_docs_with_term)
    return current_doc_tf_idf_dict

docname_to_dict_of_tf = {doc_name: compute_tf_for_a_doc(doc_ls_of_strs) for doc_name, doc_ls_of_strs in docname_to_ls_of_strs.items()}

docname_to_dict_of_tfidf = {}
for doc_name in docname_to_dict_of_tf:
    doc_tf_dict = docname_to_dict_of_tf[doc_name]
    other_docs_ls_of_strs = [docname_to_ls_of_strs[other_doc_name] for other_doc_name in docname_to_ls_of_strs if other_doc_name != doc_name]
    docname_to_dict_of_tfidf[doc_name] = compute_tf_idf_for_a_doc(doc_tf_dict, other_docs_ls_of_strs)

docname_to_dict_of_tfidf

{'neural-nets': {'proc': 0.0008494450742156192,
  'natl': 0.001229547390038742,
  'acad': 0.0010721222676661332,
  'sci': 0.0013100561934475256,
  'usavol': 0.0006118734218860276,
  'pp': 0.0008494450742156192,
  'april': 0.0003874836570436849,
  'biophysics': 0.0023514962142504557,
  'neural': 7.33360009625926e-05,
  'networks': 0.00010759856582702855,
  'and': 0.0,
  'physical': 0.006113890157051184,
  'systems': 0.001304751137610738,
  'with': 0.0,
  'emergent': 0.0028217954571005464,
  'collective': 0.006587222169742643,
  'computational': 0.0006463445255248343,
  'abilitiesassociative': 0.0006118734218860276,
  'memoryparallel': 0.0006118734218860276,
  'processingcategorizationcontent-addressable': 0.0006118734218860276,
  'memoryfail-soft': 0.0006118734218860276,
  'devices': 0.0015499346281747396,
  'j': 0.001384155215448512,
  'hopfield': 0.003059367109430138,
  'division': 0.0006574501276283089,
  'of': 0.0,
  'chemistry': 0.0009405984857001822,
  'biology': 0.000566296716143

In [118]:
# transform `docname_to_dict_of_tfidf` into a matrix
all_terms = {term for doc_dict in docname_to_dict_of_tfidf.values() for term in doc_dict.keys()}
all_terms

{'relaxation',
 'submitting',
 'contralateral',
 'blockade',
 'publishers',
 'discovers',
 'monomeric',
 'rouge',
 'parallelizable',
 'terminus',
 'retccl',
 'similarly-sized',
 'say',
 'dition',
 'ﬁlters',
 'unintelligible',
 'suis',
 'combined',
 'dont',
 'rocs',
 'analytics',
 'methodologies',
 'here',
 'lished',
 'image-omic',
 'encodes',
 'novel',
 'nerr',
 'won',
 'integrative',
 'efﬁcient',
 'ﬁ',
 'arated',
 'biennially',
 '_',
 'chris',
 'franken',
 'themselves',
 'gln',
 'classiﬁca',
 'turkers',
 'modern',
 'upjohn',
 'commu',
 'offset',
 'sharp',
 'competition',
 'exclusivity',
 'tokenizer',
 'orig',
 'nilsback',
 'rially',
 'charac',
 'schroeder',
 'tanhx',
 'decoders',
 'hospital',
 'communicate',
 'splits',
 'interresidue',
 'segmenting',
 'rie',
 'pre-trained',
 'library',
 'lgnvvvit',
 'shannonsm',
 'shaded',
 'model',
 'comparative',
 'mr',
 'national',
 'tiling',
 'pharmacovigilance',
 'short-term',
 'peters',
 'node',
 'drug-disease',
 'tohigher',
 'rectiﬁed',
 'needs

In [104]:
num_docs = len(docname_to_dict_of_tfidf)
num_terms = len(all_terms)
# ref: https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.dok_matrix.html
doc_term_mat = dok_matrix((num_docs, num_terms), dtype=np.float32)

term_to_col = {term: idx for idx, term in enumerate(all_terms)}
doc_to_row = {doc: idx for idx, doc in enumerate(docname_to_dict_of_tfidf.keys())}
for doc, tfidf_dict in docname_to_dict_of_tfidf.items():
    for term, tfidf in tfidf_dict.items():
        doc_term_mat[doc_to_row[doc], term_to_col[term]] = tfidf

In [121]:
df = pd.DataFrame(0.0, index=docname_to_dict_of_tfidf.keys(), columns=sorted(all_terms))

for doc, tfidf_dict in docname_to_dict_of_tfidf.items():
    for term, tfidf in tfidf_dict.items():
        df.at[doc, term] = tfidf
df

_        __  ________  _________  __________________  \
neural-nets    0.000000  0.000000  0.000000   0.000000            0.000000   
backprop       0.006467  0.002156  0.000719   0.001437            0.000719   
cnn            0.000000  0.000000  0.000000   0.000000            0.000000   
deep-learning  0.000000  0.000000  0.000000   0.000000            0.000000   
resnet         0.000000  0.000000  0.000000   0.000000            0.000000   
attention      0.000000  0.000000  0.000000   0.000000            0.000000   
chexnet        0.000000  0.000000  0.000000   0.000000            0.000000   
densenet       0.000000  0.000000  0.000000   0.000000            0.000000   
ecg            0.000000  0.000000  0.000000   0.000000            0.000000   
gpt-2          0.000000  0.000000  0.000000   0.000000            0.000000   
chexpert       0.000000  0.000000  0.000000   0.000000            0.000000   
efficientnet   0.000000  0.000000  0.000000   0.000000            0.000000   
pubmedbert     0.000000  0.000000  0.000000   0.000000            0.000000   
clip           0.000000  0.000000  0.000000   0.000000            0.000000   
clam           0.000000  0.000000  0.000000   0.000000            0.000000   
rosettafold    0.000000  0.000000  0.000000   0.000000            0.000000   
alphafold      0.000000  0.000000  0.000000   0.000000            0.000000   
esm-2          0.000000  0.000000  0.000000   0.000000            0.000000   
conch          0.000000  0.000000  0.000000   0.000000            0.000000   
uni            0.000000  0.000000  0.000000   0.000000            0.000000   

               ___ct_o_b_e_r___       __a       __e  _natu_r_e_v_o_l_  \
neural-nets            0.000000  0.000000  0.000000          0.000000   
backprop               0.000719  0.000719  0.000719          0.000719   
cnn                    0.000000  0.000000  0.000000          0.000000   
deep-learning          0.000000  0.000000  0.000000          0.000000   
resnet                 0.000000  0.000000  0.000000          0.000000   
attention              0.000000  0.000000  0.000000          0.000000   
chexnet                0.000000  0.000000  0.000000          0.000000   
densenet               0.000000  0.000000  0.000000          0.000000   
ecg                    0.000000  0.000000  0.000000          0.000000   
gpt-2                  0.000000  0.000000  0.000000          0.000000   
chexpert               0.000000  0.000000  0.000000          0.000000   
efficientnet           0.000000  0.000000  0.000000          0.000000   
pubmedbert             0.000000  0.000000  0.000000          0.000000   
clip                   0.000000  0.000000  0.000000          0.000000   
clam                   0.000000  0.000000  0.000000          0.000000   
rosettafold            0.000000  0.000000  0.000000          0.000000   
alphafold              0.000000  0.000000  0.000000          0.000000   
esm-2                  0.000000  0.000000  0.000000          0.000000   
conch                  0.000000  0.000000  0.000000          0.000000   
uni                    0.000000  0.000000  0.000000          0.000000   

                     _s  ...  ﬁxed-length    ﬁxing   ﬂexible       ﬂip  \
neural-nets    0.000000  ...     0.000000  0.00000  0.000000  0.000000   
backprop       0.000719  ...     0.000000  0.00000  0.000000  0.000000   
cnn            0.000000  ...     0.000000  0.00000  0.000000  0.000000   
deep-learning  0.000000  ...     0.000000  0.00000  0.000000  0.000000   
resnet         0.000000  ...     0.000000  0.00000  0.000346  0.001093   
attention      0.000000  ...     0.000000  0.00000  0.000000  0.000000   
chexnet        0.000000  ...     0.000000  0.00000  0.000000  0.000000   
densenet       0.000000  ...     0.000000  0.00000  0.000000  0.000000   
ecg            0.000000  ...     0.000000  0.00000  0.000000  0.000000   
gpt-2          0.000000  ...     0.000423  0.00000  0.000536  0.000000   
chexpert       0.000000  ...     0.

# Dimensionality Reduction

In [105]:
# normalize sparse_matrix such that each row is a unit vector
doc_term_mat_normalized = normalize(doc_term_mat, norm='l2', axis=1).todense()

In [106]:
doc_term_mat_normalized.shape

(20, 11441)

In [107]:
# perform pca on doc_term_mat_normalized
from sklearn.decomposition import PCA
pca = PCA(n_components=2) 
pca.fit_transform(np.asarray(doc_term_mat_normalized))
# plot the pca
plt.scatter(pca.components_[0], pca.components_[1])
plt.show()

In [113]:
pca.fit_transform(np.asarray(doc_term_mat_normalized))

array([[-0.01367275,  0.11110992],
       [ 0.00732936,  0.12876615],
       [-0.12191298,  0.23132513],
       [-0.02543763,  0.12226328],
       [-0.1373812 ,  0.56510484],
       [-0.04907139,  0.11376462],
       [-0.15650849, -0.00672241],
       [-0.1586404 ,  0.53457075],
       [-0.0651377 , -0.03771515],
       [-0.06049072, -0.08048552],
       [-0.12820604, -0.067113  ],
       [-0.11788166,  0.16185345],
       [-0.06487461, -0.11470247],
       [-0.16284086, -0.14923139],
       [-0.23479696, -0.3800759 ],
       [ 0.60982466, -0.05888727],
       [ 0.6759741 , -0.06904298],
       [ 0.6562079 , -0.07229223],
       [-0.22224496, -0.45654935],
       [-0.23023652, -0.47594032]], dtype=float32)

In [116]:
np.asarray(doc_term_mat_normalized).shape

(20, 11441)

In [84]:
print(pca.explained_variance_ratio_)
print(pca.components_)

[0.08993372 0.08312938]
[[-0.01235817 -0.0002263  -0.00148722 ...  0.0001975  -0.00043869
   0.00240273]
 [-0.01390827 -0.00042722  0.00110948 ...  0.000803    0.00239193
  -0.00135129]]
